# **Project Title: Risk Assessment for the Establishment and Operations Of a Movie Studio**

# ****Author: GROUP 8****

## **Step 1: Business Understanding** 
The company aims to diversify its operations by venturing into the competitive film production industry by creating original video content. However, due to lack of experience in movie production, they require data-driven insights to identify successful movie trends at the box office in the current market.

### ****Challenges****
The company lacks experience in the film industry, has uncertainity about the necessities of the industry like best genre, producer and production strategies that resonate with the audience and has no information on the financial risk associated with film idustry that it may face

### ****Proposed Solution****
Analyze historical box office data to identify information and patterns like genres, budgets, release timing, franchises in successful films. Then provide actionable recommendations to guide the company on the things to prioritize and the studio content strategy to adopt.

### ****Conclusion****
By leveraging data analysis, the company can identify successful and failure patterns inorder to adopt the right strategy, mitigate risks and align its film production with market demand.

### ****Problem Statement****
The company aims to establish a new movie studio to capitalize on the growing demand for original video content. However, due to lack of experience in the film industry, uncertainty over the type of production to venture into and lack of info on the finacial risk, the studio faces significant challenges in determining which types of movies are most likely to succeed.

### ****Objectives****
1. To analyze historical box office data to identify trends in successful films
2. To determine the relationship between production budget and revenue
3. To compare audience and critic ratings
4. To provide actionable recommendations on best film production strategies


## **Step 2: Data Undertanding**

### ****Data Source****
The folder zippedData contains historical box office data scrapped from https://www.boxofficemojo.com/, https://www.imdb.com/, https://www.rottentomatoes.com/, https://www.themoviedb.org/, https://www.the-numbers.com/ that will be used for this analysis, detailing all relevant information about film industry, including the movie genres, their budget, revenue, producers, release date and reviews.

### **column description**
1. genres

   Type: Categorical

   Description: The primary category/categories of the film (e.g., Action, Comedy, Drama, Animation).

2. production_budget

   Type: Numerical

   Description: Total production cost of the film in USD.

3. domestic_gross/worldwide_gross

   Type: Numerical

   Description: Total box office earnings in USD.

4. release_date	

   Type: Categorical

   Description: Month the film was released

5. runtime_minutes

   Type: Numerical

   Description: Duration of the film in minutes.

6. averagerating
   
   Type: Numerical
   
   Description: Audience or critic aggregate score


In [1]:
# import relevant libraries
import pandas as pd
import numpy as np
import sqlite3 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import zipfile
import tempfile
import os

In [2]:
# loading the datasets
bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
rt_info = pd.read_csv("zippedData/rt.movie_info.tsv.gz", sep='\t')
rt_reviews = pd.read_csv("zippedData/rt.reviews.tsv.gz", sep='\t', encoding='latin1')
movie_DB = pd.read_csv("zippedData/tmdb.movies.csv.gz")
numbers = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")

# Extract the .db file from the ZIP
# with zipfile.ZipFile("zippedData/im.db.zip", 'r') as zip_ref:
#     # Extract to subfolder
#     zip_ref.extract("im.db", path="zippedData/extracted")  

# Extract the .db file from the ZIP
with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
    with zipfile.ZipFile("zippedData/im.db.zip", 'r') as zip_ref:
        tmp_file.write(zip_ref.read("im.db"))

# # Connect to the extracted database
conn = sqlite3.connect(tmp_file.name)  # Path to extracted file

# Run your query
q = """
SELECT *
FROM movie_basics
JOIN movie_ratings USING (movie_id)
"""
imdb = pd.read_sql(q, conn)

# Close connection
conn.close()

In [3]:
# Display the first 5 rows
bom.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [4]:
# get info summary of the dataset
bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


Box office mojo dataset has 3387 rows. From this, it can be seen that the columns studio, domestic_gross and foreign_gross have missing values.

In [5]:
# Display the first 5 rows
imdb.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119


In [6]:
# get info summary of the dataset
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         73856 non-null  object 
 1   primary_title    73856 non-null  object 
 2   original_title   73856 non-null  object 
 3   start_year       73856 non-null  int64  
 4   runtime_minutes  66236 non-null  float64
 5   genres           73052 non-null  object 
 6   averagerating    73856 non-null  float64
 7   numvotes         73856 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 4.5+ MB


From the movie_basics and movie_ratings tables form IMDB sqlite file extracted, the final joined contains 73856 rows. It can be seen that columns runtime_minutes and genres contains missing values

In [7]:
# Display the first 5 rows
numbers.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [8]:
# get info summary of the dataset
numbers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


## Step 3: Data Preparations